**Libraries**

In [131]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import arff
from ydata_profiling import ProfileReport

## Datenvorbereitung

**Daten sind zu importieren**

In [106]:
data_freq = arff.load('data/freMTPL2freq.arff')
df_freq = pd.DataFrame(data_freq, columns=["idpol", "claimnb", "exposure", "area", "vehpower",
"vehage","drivage", "bonusmalus", "vehbrand", "vehgas", "density", "region"])
data_sev = arff.load('data/freMTPL2sev.arff') 
df_sev = pd.DataFrame(data_sev, columns=["idpol", "claimamount"])

In [107]:
#TODO: mtpl2 #3 - Erster Überblick über den Datensatz schaffen.

**Automatisierter Datenreport wird erstellt**

In [108]:
""" profile_sev = ProfileReport(df_sev, title="Profiling Report df_sev")
profile_freq = ProfileReport(df_freq, title="Profiling Report df_freq")

#Profilreport wird als HTML abgelegt

profile_sev.to_file("data/profiling-df_sev.html")
profile_freq.to_file("data/profiling-df_freq.html") """

' profile_sev = ProfileReport(df_sev, title="Profiling Report df_sev")\nprofile_freq = ProfileReport(df_freq, title="Profiling Report df_freq")\n\n#Profilreport wird als HTML abgelegt\n\nprofile_sev.to_file("data/profiling-df_sev.html")\nprofile_freq.to_file("data/profiling-df_freq.html") '

#### Erste Findings aus Profilreport

Erste kritische Findings in df_sev:

* claimamount ist stark rechtschief.
* Es gibt 241 Duplikate.

Erste kritische Findings in df_freq:

* drivage is highly overall correlated with bonusmalus	High correlation  
* bonusmalus is highly overall correlated with drivage	High correlation  
* density is highly overall correlated with area	High correlation  
* area is highly overall correlated with density	High correlation  
* idpol has unique values	Unique  
* claimNb has 643953 (95.0%) zeros	Zeros  
* vehage has 57739 (8.5%) zeros  

#### Überblick über Dimensionen

In [109]:
print("df_sev hat " +str(df_sev.shape[0]) +" Reihen mit " + str(df_sev.shape[1]) + " Spalten.")

df_sev hat 26639 Reihen mit 2 Spalten.


In [110]:
print("df_freq hat " +str(df_freq.shape[0]) +" Reihen mit " + str(df_freq.shape[1]) + " Spalten.")

df_freq hat 678013 Reihen mit 12 Spalten.


#### Überblick über Datentypen

In [111]:
print(df_sev.info(), df_freq.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26639 entries, 0 to 26638
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   idpol        26639 non-null  float64
 1   claimamount  26639 non-null  float64
dtypes: float64(2)
memory usage: 416.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   idpol       678013 non-null  float64
 1   claimnb     678013 non-null  float64
 2   exposure    678013 non-null  float64
 3   area        678013 non-null  object 
 4   vehpower    678013 non-null  float64
 5   vehage      678013 non-null  float64
 6   drivage     678013 non-null  float64
 7   bonusmalus  678013 non-null  float64
 8   vehbrand    678013 non-null  object 
 9   vehgas      678013 non-null  object 
 10  density     678013 non-null  float64
 11  region      678013 n

Übersicht **df_sev**:

| Column    | Featurename | Inhalt | Datentyp (ist) | Datentyp (soll) |
|-----------|-------------|--------|--------|-----------------|
| 0         |  idpol      | ID des Vertrags | float64 | $\textcolor{red}{integer}$ |
| 1         |  claimamount| Höhe Schadensaufwände | float64 | float64 |

Übersicht **df_freq**:

| Column    | Featurename | Inhalt         | Datentyp (ist) | Datentyp (soll) |
|-----------|-------------|-----------------|--------|-----------------|
| 0         |  idpol      | ID des Vertrags | float64 | $\textcolor{red}{integer}$ |
| 1         |  claimnb    | Anzahl Schäden | float64 | $\textcolor{red}{integer}$ |
| 2         |  exposure   | Länge Versicherungszeitraum | float64 | float64 |
| 3         |  area       | Area Code | categorical | categorical | 
| 4         |  vehpower   | Kfz-Leistung | float64 | $\textcolor{red}{integer}$ |
| 5         |  vehage     | Kfz-Alter | float64 | $\textcolor{red}{integer}$ |
| 6         |  drivage    | Alter der Versicherungsnehmer | float64 | $\textcolor{red}{integer}$ |
| 7         |  bonusmalus | Schadenfreiheitsrabatt  | float64 | $\textcolor{red}{integer}$ |
| 8         |  vehbrand   | Kfz-Marke | categorical | categorical |
| 9         |  vehgas     | Kfz-Antrieb  | categorical | categorical |
| 10        |  density    | Bevölkerungsdichte in Wohnort | float64 | $\textcolor{red}{integer}$ |
| 11        |  region     | Wohnregion | categorical | categorical |

**Entfernen unnötiger Anführungszeichen**

Die Werte in **df_freq[["area","vehbrand","region"]]** führen unnötige Anführungszeichen. Daher werden die Werte in Stringformat übertragen, die Zeichen entfernt und die Werte wieder in den Category-Typen zurückübertragen.

In [112]:
df_freq.head(5)

,idpol,claimnb,exposure,area,vehpower,vehage,drivage,bonusmalus,vehbrand,vehgas,density,region
0,1.0,1.0,0.10,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
1,3.0,1.0,0.77,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
2,5.0,1.0,0.75,'B',6.0,2.0,52.0,50.0,'B12',Diesel,54.0,'R22'
3,10.0,1.0,0.09,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'
4,11.0,1.0,0.84,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'


In [113]:
temp_list = ["area","vehbrand","region"]

for col in temp_list:
    df_freq[col] = df_freq[col].astype(str)
    df_freq[col] = df_freq[col].replace("'","", regex=True)
    df_freq[col] = df_freq[col].astype("category")

df_freq[temp_list].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   area      678013 non-null  category
 1   vehbrand  678013 non-null  category
 2   region    678013 non-null  category
dtypes: category(3)
memory usage: 1.9 MB


In [114]:
display(df_freq.head(3))

,idpol,claimnb,exposure,area,vehpower,vehage,drivage,bonusmalus,vehbrand,vehgas,density,region
0,1.0,1.0,0.10,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82
1,3.0,1.0,0.77,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82
2,5.0,1.0,0.75,B,6.0,2.0,52.0,50.0,B12,Diesel,54.0,R22


**Anpassung der Datentypen**

*integer soll werden: idpol*

In [115]:
temp_list = ["idpol"]

In [116]:
for col in temp_list:
    df_freq[col] = df_freq[col].astype(int)
    df_sev[col] = df_sev[col].astype(int)

In [117]:
#Prüfung
print("df_sev: \n",type(df_sev["idpol"][0]))
print("df_freq: \n",type(df_freq["idpol"][0]))

df_sev: 
 <class 'numpy.int64'>
df_freq: 
 <class 'numpy.int64'>


*integer sollen werden: 'claimnb', 'vehpower', 'vehage', drivage', 'bonusmalus','density'*

In [118]:
temp_list = ['claimnb', 'vehpower', 'vehage', 'drivage', 'bonusmalus','density']

In [119]:
for col in temp_list:
    df_freq[col] = df_freq[col].astype(int)

In [120]:
#Prüfung
df_freq[temp_list].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   claimnb     678013 non-null  int64
 1   vehpower    678013 non-null  int64
 2   vehage      678013 non-null  int64
 3   drivage     678013 non-null  int64
 4   bonusmalus  678013 non-null  int64
 5   density     678013 non-null  int64
dtypes: int64(6)
memory usage: 31.0 MB


**Duplikate-Handling**

df_sev hat einige Duplikate. Einigen Vertragsnummern wurden mehrere Schadensaufwendungen zugeordnet. Wir schauen uns das genauer an.

In [147]:
# Ein erster Blick
check_dupl_head = df_sev[df_sev.duplicated(keep=False)].sort_values('idpol').head(5)
check_dupl_tail = df_sev[df_sev.duplicated(keep=False)].sort_values('idpol').tail(5)
display(check_dupl_head, check_dupl_tail)

,idpol,claimamount
5022,18698,1128.12
6355,18698,1128.12
4521,19887,1128.12
4514,19887,1128.12
6322,24992,1128.12


,idpol,claimamount
675,5027513,1204.0
1524,5069017,1204.0
1550,5069017,1204.0
2093,6097196,660.0
2003,6097196,660.0


In [187]:
df_sev_duplicates = pd.DataFrame(df_sev[df_sev.duplicated(keep=False)].value_counts().reset_index())
display(df_sev_duplicates.sort_values("count", ascending=False))

,idpol,claimamount,count
0,2262511,1500.00,6
2,2282134,1500.00,4
1,93954,1128.12,4
3,2227533,1500.00,3
5,2282134,10000.00,3
...,...,...,...
89,4165911,1204.00,2
90,4170413,1204.00,2
91,4181834,1204.00,2
92,4137419,1204.00,2


Sind alle Vertragsnummern immer mit dem gleichen Schaden versehen?

In [193]:
df_sev_duplicates[df_sev_duplicates["idpol"].duplicated(keep=False)].value_counts()

idpol    claimamount  count
2227533  83.12        2        1
         1500.00      3        1
         7500.00      2        1
2262511  1000.00      2        1
         1500.00      6        1
         10000.00     2        1
2282134  1500.00      4        1
         10000.00     3        1
2286775  1500.00      2        1
         10000.00     2        1
3254353  1000.00      2        1
         1200.00      2        1
         1800.00      2        1
Name: count, dtype: int64

Wir sehen also, dass zum Beispiel der Vertrag 2227533
* zweimal einen Betrag von 83,12, 
* dreimal einen Betrag von 1500,00 und
* zweimal einen Betrag von 7500,00 abgerufen hat.

Das Ergibt eine Summe von 19666,24

Wie viele Schadensfälle sind dem Vertrag 2227533 denn zugeordnet?

In [197]:
df_freq.query("idpol == 2227533")

,idpol,claimnb,exposure,area,vehpower,vehage,drivage,bonusmalus,vehbrand,vehgas,density,region


Die Vertragsnummer **2227533** scheint nicht zu existieren obwohl ihr Schadensaufwendungen in Höhe von knapp 20000 zugeordnet sind.  
Das muss geklärt werden!

Wie sieht es zum Beispiel mit Vertrag 3254353 aus?

Der Vertrag 3254353 hat nach unseren Daten
* zweimal einen Betrag von 1000,00,
* zweimal einen Betrag von 1200,00 und
* zweimal einen Betrag von 1800,00 abgerufen.

Das ergibt eine Summe von 8000

In [198]:
df_freq.query("idpol == 3254353")

,idpol,claimnb,exposure,area,vehpower,vehage,drivage,bonusmalus,vehbrand,vehgas,density,region
488270,3254353,11,0.07,D,4,13,53,50,B1,Regular,824,R91


Der Vertragsnummer 3254353 wurden 11 Schadensfälle zugeordnet.  
Auf der anderen seite wurden 6 Beträge diesen 11 Schadensfällen zugeordnet.

Schauen wir uns die Vertragsnummer 6097196 an.

In [207]:
display(df_sev_duplicates.sort_values("count", ascending=False).query('idpol == 6097196'))

,idpol,claimamount,count
240,6097196,660.0,2


Der Vertragsnummer 6097196 werden zwei Beträge von 660 zugeordnet.  

Wie viele Schadensfälle stehen dem gegenüber?

In [208]:
df_freq.query("idpol == 6097196")

,idpol,claimnb,exposure,area,vehpower,vehage,drivage,bonusmalus,vehbrand,vehgas,density,region
667008,6097196,2,0.42,E,8,2,52,64,B12,Diesel,3317,R93


Beim Vertrag 6097196 stehen die gleiche Anzahl von Schäden den Beträgen gegenüber.

**Zwischenstand:**

Die Datenlage sollte auf jeden Fall noch einmal vertieft betrachtet werden. 
Es gibt wohl nach erster Analyse 

* Schadensaufwendungen, die keinen Vertragsnehmern zugeordnet werden können,
* Vertragsnehmer, die Schadensfälle gemeldet haben, denen keine Schadensaufwendungen zugeordnet sind und
* Schadensaufwendungen, die mehrmals gebucht wurden.

Aus Gründen der Angemessenheit fahre ich nun fort.

**Wie also nun umgehen mit den Duplikaten in df_sev ?**

Ich entscheide mich dafür, die Duplikate auf die Vertragsnummern aufzusummieren.

In [219]:
# Gruppieren nach 'idpol' und Summierung der 'claimamount' Werte
summed_claims = df_sev.groupby('idpol')['claimamount'].sum().reset_index()
df_summed_claims = pd.DataFrame(summed_claims)

Hat Vertragsnummer 6097196 nun einen Claimamount von 1320?

In [218]:
df_summed_claims.query('idpol == 6097196')

,idpol,claimamount
24747,6097196,1320.0


In [79]:
#TODO: Wie soll das merging gemacht werden? Warum diese Entscheidung?
#TODO: nach dem Merging: Missing Values?